In [29]:
! pip install scikeras

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


### 1. Preparações

In [2]:
dataset = pd.read_csv("/kaggle/input/santander-customer-satisfaction/train.csv")

In [4]:
COLUNAS_CONSTANTES = [coluna for coluna in dataset.columns if dataset[coluna].nunique() == 1]
dataset = dataset.drop(columns = COLUNAS_CONSTANTES)
dataset.shape

(76020, 337)

In [5]:
ID = dataset.ID
TARGET = dataset.TARGET

dataset.drop(columns = "ID", inplace = True)
dataset.drop(columns = "TARGET", inplace = True)

In [6]:
dataset.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016


#### 1.1 Escalando o Conjunto

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dataset_scaled = scaler.fit_transform(dataset.copy())

dataset_scaled = pd.DataFrame(dataset_scaled, columns = dataset.columns)
dataset_scaled.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


#### 1.2 Undersampling

In [8]:
from imblearn.under_sampling import RandomUnderSampler

SEED = 912039

In [9]:
sampler = RandomUnderSampler(random_state = SEED)
x_under, y_under = sampler.fit_resample(dataset_scaled, TARGET)
dataset_under = pd.concat([x_under, y_under], axis = 1)
dataset_under.TARGET.value_counts()

TARGET
0    3008
1    3008
Name: count, dtype: int64

In [10]:
y_undersampling = dataset_under.TARGET
dataset_under.drop("TARGET", axis = 1, inplace = True)

#### 1.3 Treino e Teste

In [16]:
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(dataset_under, y_undersampling, 
                                                        test_size=0.2, random_state=SEED,
                                                        stratify = y_undersampling)

x_treino = pd.DataFrame(x_treino, columns = dataset.columns)
x_teste = pd.DataFrame(x_teste, columns = dataset.columns)

In [17]:
y_treino.value_counts()

TARGET
1    2406
0    2406
Name: count, dtype: int64

In [18]:
y_teste.value_counts()

TARGET
0    602
1    602
Name: count, dtype: int64

### 2. Treinamento

In [44]:
from tensorflow import keras


modelo = keras.Sequential([
    keras.layers.Input(shape=dataset.shape[1:]),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
    
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [45]:
treinamento = modelo.fit(x_treino, y_treino, epochs=10, batch_size=32, validation_data=(x_teste, y_teste))
resultado = modelo.evaluate(x_teste, y_teste)
print(f"Acurácia no conjunto de teste: {resultado[1]:.2f}")

Epoch 1/10
151/151 [==============================] - 2s 5ms/step - loss: 0.6146 - accuracy: 0.6845 - val_loss: 0.6224 - val_accuracy: 0.6769
Epoch 2/10
151/151 [==============================] - 0s 3ms/step - loss: 0.5886 - accuracy: 0.6939 - val_loss: 0.5859 - val_accuracy: 0.6852
Epoch 3/10
151/151 [==============================] - 0s 3ms/step - loss: 0.5687 - accuracy: 0.7066 - val_loss: 0.5722 - val_accuracy: 0.6902
Epoch 4/10
151/151 [==============================] - 0s 3ms/step - loss: 0.5566 - accuracy: 0.7143 - val_loss: 0.5655 - val_accuracy: 0.6935
Epoch 5/10
151/151 [==============================] - 0s 3ms/step - loss: 0.5477 - accuracy: 0.7244 - val_loss: 0.5657 - val_accuracy: 0.6927
Epoch 6/10
151/151 [==============================] - 0s 3ms/step - loss: 0.5404 - accuracy: 0.7361 - val_loss: 0.5540 - val_accuracy: 0.7259
Epoch 7/10
151/151 [==============================] - 0s 3ms/step - loss: 0.5351 - accuracy: 0.7398 - val_loss: 0.5511 - val_accuracy: 0.7259
Epoch 

#### 2.1 Otimizando a Rede

In [53]:
import tensorflow as tf

def construir_modelo(aprendizado=0.001, taxa_dropout=0.3, unidades=[128, 64, 32], regularizador_kernel=0.01):
    modelo = keras.Sequential()
    modelo.add(keras.layers.Input(shape=dataset.shape[1:]))
    
    for unidade in unidades:
        modelo.add(keras.layers.Dense(unidade, activation='relu', kernel_regularizer=keras.regularizers.l2(regularizador_kernel)))
        modelo.add(keras.layers.BatchNormalization())
        modelo.add(keras.layers.Dropout(taxa_dropout))
    
    modelo.add(keras.layers.Dense(1, activation='sigmoid'))
    
    otimizador = keras.optimizers.Adam(learning_rate=aprendizado)
    modelo.compile(optimizer=otimizador, loss='binary_crossentropy', metrics=['accuracy'])
    
    return modelo

In [57]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

parametros = {
    'aprendizado': [0.0001, 0.001, 0.01, 0.1],
    'taxa_dropout': [0.2, 0.3, 0.4, 0.5],
    'unidades': [(128, 64, 32), (256, 128, 64), (64, 64, 64), (512, 256, 128), (512, 128, 32),
                 (128, 128, 128)],
    'regularizador_kernel': [0.1, 0.01, 0.001, 0.0001]
}

modelo = KerasClassifier(build_fn=construir_modelo, epochs=20, batch_size=32, verbose=0)
busca = RandomizedSearchCV(estimator=modelo, param_distributions=parametros, cv=5, random_state = SEED,
                           n_iter = 10)

resultado = busca.fit(x_treino, y_treino)

# Exiba os melhores hiperparâmetros
print("Melhor combinação de hiperparâmetros: ", resultado.best_params_)

# Avalie o desempenho do modelo no conjunto de validação
melhor_modelo = resultado.best_estimator_
validacao = melhor_modelo.score(x_teste, y_teste)
print("Acurácia no conjunto de validação: ", validacao)


/tmp/ipykernel_32/2273794367.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  modelo = KerasClassifier(build_fn=construir_modelo, epochs=20, batch_size=32, verbose=0)


Melhor combinação de hiperparâmetros:  {'unidades': (128, 128, 128), 'taxa_dropout': 0.3, 'regularizador_kernel': 0.001, 'aprendizado': 0.001}
Acurácia no conjunto de validação:  0.7084717750549316


#### 2.2 Modelo Final

In [58]:
modelo = keras.Sequential([
    keras.layers.Input(shape=dataset.shape[1:]),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])

otimizador = keras.optimizers.Adam(learning_rate=0.001)

modelo.compile(optimizer=otimizador, loss='binary_crossentropy', metrics=['accuracy'])

In [59]:
treinamento = modelo.fit(x_treino, y_treino, epochs=20, batch_size=32, validation_data=(x_teste, y_teste))
resultado = modelo.evaluate(x_teste, y_teste)
print(f"Acurácia no conjunto de teste: {resultado[1]:.2f}")

Epoch 1/20
151/151 [==============================] - 4s 6ms/step - loss: 1.0752 - accuracy: 0.6266 - val_loss: 0.9683 - val_accuracy: 0.6877
Epoch 2/20
151/151 [==============================] - 1s 5ms/step - loss: 0.9557 - accuracy: 0.6663 - val_loss: 0.9040 - val_accuracy: 0.6836
Epoch 3/20
151/151 [==============================] - 1s 5ms/step - loss: 0.9040 - accuracy: 0.6744 - val_loss: 0.8602 - val_accuracy: 0.6935
Epoch 4/20
151/151 [==============================] - 1s 6ms/step - loss: 0.8613 - accuracy: 0.6991 - val_loss: 0.8315 - val_accuracy: 0.6952
Epoch 5/20
151/151 [==============================] - 1s 5ms/step - loss: 0.8320 - accuracy: 0.7030 - val_loss: 0.8147 - val_accuracy: 0.6944
Epoch 6/20
151/151 [==============================] - 1s 5ms/step - loss: 0.8008 - accuracy: 0.7099 - val_loss: 0.7986 - val_accuracy: 0.6960
Epoch 7/20
151/151 [==============================] - 1s 5ms/step - loss: 0.7751 - accuracy: 0.7147 - val_loss: 0.7480 - val_accuracy: 0.7267
Epoch 

### 3. Resultado

In [60]:
alvo = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")

alvo_id = alvo.ID
alvo.drop(columns = "ID", inplace = True)

scaler = MinMaxScaler()

copia = alvo.copy()
copia = scaler.fit_transform(copia)

alvo = pd.DataFrame(copia, columns = alvo.columns)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [61]:
alvo = alvo.drop(columns = COLUNAS_CONSTANTES)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [63]:
modelo.fit(dataset_under, y_undersampling, epochs=20, batch_size=32, validation_data=(x_teste, y_teste))

Epoch 1/20
188/188 [==============================] - 1s 6ms/step - loss: 0.6077 - accuracy: 0.7271 - val_loss: 0.6047 - val_accuracy: 0.7267
Epoch 2/20
188/188 [==============================] - 1s 5ms/step - loss: 0.6017 - accuracy: 0.7267 - val_loss: 0.5889 - val_accuracy: 0.7118
Epoch 3/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5965 - accuracy: 0.7272 - val_loss: 0.5823 - val_accuracy: 0.7434
Epoch 4/20
188/188 [==============================] - 1s 6ms/step - loss: 0.5873 - accuracy: 0.7354 - val_loss: 0.6050 - val_accuracy: 0.7292
Epoch 5/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5911 - accuracy: 0.7284 - val_loss: 0.5809 - val_accuracy: 0.7243
Epoch 6/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5875 - accuracy: 0.7261 - val_loss: 0.5843 - val_accuracy: 0.7342
Epoch 7/20
188/188 [==============================] - 1s 5ms/step - loss: 0.5822 - accuracy: 0.7311 - val_loss: 0.5740 - val_accuracy: 0.7184
Epoch 

In [71]:
resultados = modelo.predict(alvo)
resposta = pd.concat([alvo_id, pd.Series(resultados.flatten(), name = "TARGET")], axis = 1)
resposta

2370/2370 [==============================] - 5s 2ms/step


,ID,TARGET
0,2,0.415108
1,5,0.464581
2,6,0.199320
3,7,0.432761
4,9,0.207681
...,...,...
75813,151831,0.412754
75814,151832,0.291524
75815,151833,0.248818
75816,151834,0.433497


In [85]:
binary_predictions = (resultados > 0.5).astype(int)
binary_predictions[432]

array([0])

In [89]:
resposta = pd.concat([alvo_id, pd.Series(binary_predictions.flatten(), name = "TARGET")], axis = 1)
resposta

,ID,TARGET
0,2,0
1,5,0
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


In [90]:
resposta.TARGET.value_counts()

TARGET
0    57503
1    18315
Name: count, dtype: int64

In [91]:
resposta.to_csv("resposta.csv", index = False)